In [1]:
import kfp
import kfp.dsl as dsl
import kfp.onprem as onprem

In [2]:
def echo_op(text):
    return dsl.ContainerOp(
        name='echo',
        image='library/bash:4.4.23',
        command=['sh', '-c'],
        arguments=['echo "$0"', text],
    )


In [3]:
@dsl.pipeline(
    name='FMnistPipeline',
    description='mnist '
)
def fmnist_pipeline(learning_rate, dropout_rate, epoch, act, 
                    layer, checkpoint_dir, saved_model_dir, tensorboard_log):
    
    exit_task = echo_op("Done!")
    
    with dsl.ExitHandler(exit_task): 
        
        kubeflow_pvc = dsl.PipelineVolume(pvc="kubeflow-pvc", name="kubeflow-pvc")
        
        mnist = dsl.ContainerOp(
            name='FMnist',
            image='kubeflow-registry.default.svc.cluster.local:30000/katib-job:5FEC6E94',
            command=['python', '/app/fmnist-save-model.py'],
            arguments=[
                "--learning_rate", learning_rate,
                "--dropout_rate", dropout_rate,
                "--epoch", epoch,
                "--act", act,
                "--layer", layer,
                "--checkpoint_dir", checkpoint_dir,
                "--saved_model_dir", saved_model_dir,
                "--tensorboard_log", tensorboard_log
            ],
            pvolumes={"/result": kubeflow_pvc}
        ).set_gpu_limit(1)
        
        result = dsl.ContainerOp(
            name='list_list',
            image='library/bash:4.4.23',
            command=['ls', '-R', '/result'],
            pvolumes={"/result": mnist.pvolume}
        )

        result.after(mnist)

In [4]:
arguments = {'learning_rate': '0.001397',
             'dropout_rate': '0.18',
             'epoch' : '11',
             'act' : 'sigmoid',
             'layer': '2',
             'checkpoint_dir': '/reuslt/training_checkpoints',
             'saved_model_dir':'/result/saved_model',
             'tensorboard_log': '/result/log' 
            }

In [5]:
if __name__ == '__main__':
    kfp.Client().create_run_from_pipeline_func(pipeline_func=fmnist_pipeline, 
                                               arguments=arguments)